Twitter Data Extraction and Sentiment Analysis

In [ ]:
!pip install snscrape

import datetime as dt
import snscrape.modules.twitter as sntwitter
import pandas as pd
import re

#Get user input
query = input("Query: ")

#As long as the query is valid (not empty or equal to '#')...
if query != '':
    noOfTweet = input("Enter the number of tweets you want to Analyze: ")
    #if noOfTweet != '' :
    noOfDays = input("Enter the number of days you want to Scrape Twitter for: ")
    if noOfDays != '':
    #Creating list to append tweet data
        tweets_list = []
        #now = dt.date.today()
        now=pd.to_datetime("2012-12-31")
        now = now.strftime('%Y-%m-%d')
        #yesterday = dt.date.today() - dt.timedelta(days = int(noOfDays))
        yesterday=pd.to_datetime(now) - dt.timedelta(days = int(noOfDays))
        yesterday = yesterday.strftime('%Y-%m-%d')
        d=yesterday
        for i in range(int(noOfDays)):
            d=(pd.to_datetime(d) + dt.timedelta(days = 1)).strftime('%Y-%m-%d')
            n=(pd.to_datetime(d) + dt.timedelta(days = 1)).strftime('%Y-%m-%d')
            for i,tweet in enumerate(sntwitter.TwitterSearchScraper(query + ' lang:en since:' +  d + ' until:' + n + ' -filter:links -filter:replies').get_items()):
                if i > int(noOfTweet):
                    break
                tweets_list.append([tweet.date.strftime("%Y-%m-%d"),tweet.date.strftime("%H:%M:%S"), tweet.id, tweet.content, tweet.username,'TWITTER'])
                
        #Creating a dataframe from the tweets list above 
        df = pd.DataFrame(tweets_list, columns=['Date','Time', 'Tweet Id', 'Text', 'Username','Source'])
        
df

# Create a function to clean the tweets
def cleanTxt(text):
    text = re.sub('@[A-Za-z0–9]+', '', text) #Removing @mentions
    text = re.sub('#', '', text) # Removing '#' hash tag
    text = re.sub('RT[\s]+', '', text) # Removing RT
    text = re.sub('https?:\/\/\S+', '', text) # Removing hyperlink
    return text

#applying this function to Text column of our dataframe
df["Text"] = df["Text"].apply(cleanTxt)


df= df[["Date","Time","Text"]]




Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 59 kB 3.9 MB/s 
Query: womens safety dwarka
Enter the number of tweets you want to Analyze: 100
Enter the number of days you want to Scrape Twitter for: 366


In [ ]:
df.to_csv('Twitter.csv', mode='a', index = False, header=None)

In [ ]:
import pandas as pd
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from datetime import datetime
from google.colab import drive
drive.mount('/content/drive')

data =pd.read_csv("/content/Twitter.csv")
data.head()

import nltk
nltk.download('vader_lexicon')
vader=SentimentIntensityAnalyzer()

compond=lambda summary: vader.polarity_scores(summary)['compound']
data['Compound']=data['SUMMARY'].apply(compond)

negative=lambda summary: vader.polarity_scores(summary)['neg']
data['Negative']=data['SUMMARY'].apply(negative)

neutral=lambda summary: vader.polarity_scores(summary)['neu']
data['Neutral']=data['SUMMARY'].apply(neutral)

positive=lambda summary: vader.polarity_scores(summary)['pos']
data['Positive']=data['SUMMARY'].apply(positive)
data


data.to_csv('Twitter_output.csv',mode='a',index = False, header=None)


#Sentiment Analysis
def percentage(part,whole):
    return 100 * float(part)/float(whole)

#Assigning Initial Values
positive = 0
negative = 0
neutral = 0
#Creating empty lists
summary_list = []
neutral_list = []
negative_list = []
positive_list = []


#Iterating over the Summary in the dataframe
for summary in data['SUMMARY']:
    summary_list.append(summary)
    analyzer = SentimentIntensityAnalyzer().polarity_scores(summary)
    neg = analyzer['neg']
    neu = analyzer['neu']
    pos = analyzer['pos']
    comp = analyzer['compound']
    
    if neg > pos:
        negative_list.append(summary) #appending the tweet that satisfies this condition
        negative += 1 #increasing the count by 1
    elif pos > neg:
        positive_list.append(summary) #appending the tweet that satisfies this condition
        positive += 1 #increasing the count by 1
    elif pos == neg:
        neutral_list.append(summary) #appending the tweet that satisfies this condition
        neutral += 1 #increasing the count by 1 

positive = percentage(positive, len(data)) #percentage is the function defined above
negative = percentage(negative, len(data))
neutral = percentage(neutral, len(data))


#Converting lists to pandas dataframe
summary_list = pd.DataFrame(summary_list)
neutral_list = pd.DataFrame(neutral_list)
negative_list = pd.DataFrame(negative_list)
positive_list = pd.DataFrame(positive_list)

#using len(length) function for counting
print("Last 365 days, there have been", len(summary_list) ,  "tweets/news on women's safety in Delhi" ,  end='\n*')
print("Positive Sentiment:", '%.2f' % len(positive_list), end='\n*')
print("Neutral Sentiment:", '%.2f' % len(neutral_list), end='\n*')
print("Negative Sentiment:", '%.2f' % len(negative_list), end='\n*')




Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


KeyError: ignored